<a href="https://colab.research.google.com/github/tsakailab/prml/blob/master/ipynb/ex_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 回帰分析（regression analysis）

----

氏名：

学生番号：

----
基本課題（必須）

    1. Example 1 と Example 2について多項式回帰を実行し，それぞれ適切な次数（degree）について考察してください．
       また，データに標準化は回帰にどのような効果があるか観察してください．

（ここに回答を書いてください）



    2. 「最小二乗解の様々な計算方法」の「★直接法」を実装してください．また，計算時間を %timeit で計り，比較してください．

（ここに回答を書いてください）参考：[timeit](https://docs.python.org/3/library/timeit.html)



    3. 「最小二乗解の様々な計算方法」の「☆反復法」の関数 lstsq_gd を完成させ，データや回帰モデル，標準化の有無，
       学習率の設定などによる振る舞いの違いを報告してください．

（ここに回答を書いてください）



    4.その他，気づいたこと，調べたことを書いてください．

（ここに回答を書いてください）



----
発展課題（任意）がこのノートブックの後半にあります．

### 単回帰または多項式回帰用のデータを生成します．
- Example 1～4 からひとつ選んで実行してください．

In [ ]:
# Example 1: manually defined
import numpy as np
train = [ np.array([5., 3., 0., 4.]), np.array([4., 4., 1., 3.]) ]
val = [ np.array([1.0,  2., 2.5, 4.]), np.array([1.5, 2., 3.5, 4.]) ]

In [ ]:
# Example 2: artificial dataset
# http://www.scipy-lectures.org/packages/scikit-learn/auto_examples/plot_bias_variance.html#bias-and-variance-of-polynomial-fit
import numpy as np
generating_func = lambda x, err=0.5: np.random.normal(10 - 1. / (x + 0.1), err)
n_samples = 300
np.random.seed(0)
train = [10 ** np.linspace(-2, 0, n_samples), None]
train[1] = generating_func(train[0])
val = [np.linspace(0, 1.0, 100), None]
val[1] = generating_func(val[0])

In [ ]:
# Example 3: make_regression
from sklearn.datasets import make_regression
train = make_regression(
    n_samples=20, n_features=1, random_state=0, noise=8.0, bias=10.0
)
val = make_regression(
    n_samples=20, n_features=1, random_state=0, noise=3.0, bias=10.0
)

In [ ]:
# Example 4: the diabetes dataset
import numpy as np
from sklearn import datasets
diabetes = datasets.load_diabetes()
diabetes.target /= diabetes.target.max()
# Use only one feature
train = [ diabetes.data[-20:, np.newaxis, 2].ravel(), diabetes.target[-20:] ]
val = [ diabetes.data[:-20, np.newaxis, 2].ravel(), diabetes.target[:-20] ]

In [ ]:
#@title 関数 plot_reg を定義します（理解不要）．
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures

npopts = lambda: np.printoptions(threshold=16,edgeitems=4)

def plot_reg(train, test=None, w=None, degree=1):
    xmin, xmax, dx = train[0].min(), train[0].max(), (train[0].max()-train[0].min())*0.2
    ymin, ymax, dy = train[1].min(), train[1].max(), (train[1].max()-train[1].min())*0.2

    if test is not None:
        xmin, xmax, dx = min(xmin, test[0].min()), max(xmax, test[0].max()), max(dx, (test[0].max()-test[0].min())*0.2)
        ymin, ymax, dy = min(ymin, test[1].min()), max(ymax, test[1].max()), max(dy, (test[1].max()-test[1].min())*0.2)

    plt.figure(figsize=(3, 3.5))
    ax = plt.axes()
    ax.scatter(train[0], train[1], marker='x', c='darkviolet', s=20, zorder=10)
    if test is not None:
        ax.scatter(test[0], test[1], marker='o', c='0.75', s=10)
    if w is not None:
        x_regr = np.linspace(xmin-dx, xmax+dx, 100)
        X_regr = PolynomialFeatures(degree).fit_transform(x_regr[:, np.newaxis])
        y_regr = X_regr.dot(w)
        ax.plot(x_regr, y_regr, 'b-', zorder=20)

    ax.set_xlim(xmin-dx, xmax+dx)
    ax.set_ylim(ymin-dy, ymax+dy)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.tight_layout()

In [ ]:
#@title 訓練データ `x`, `y` と検証データ `x_val`, `y_val` を設定し，プロットします．<p>訓練データ（$\times$），検証データ（$\bullet$）</p>
x, y = train[0][:,np.newaxis], train[1]
x_val, y_val = val[0][:,np.newaxis], val[1]
plot_reg((x, y), (x_val, y_val))

In [ ]:
#@title このセルは必要に応じて実行してください．<br>[sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)による標準化で x と y の平均をゼロにします．
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

xy = scaler.fit_transform(np.c_[train[0],train[1]])
x, y = xy[:,0][:,np.newaxis], xy[:,1]

xy = scaler.transform(np.c_[val[0],val[1]])
x_val, y_val = xy[:,0][:,np.newaxis], xy[:,1]

plot_reg((x, y), (x_val, y_val))

### scikit-learnを使う方法
- 線形回帰：[sklearn.linear_model.LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
- 多項式回帰の計画行列の作成：[sklearn.preprocessing.PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)
- 一連の処理からなるモデルの定義：[sklearn.pipeline.Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

In [ ]:
# 単回帰
from sklearn import linear_model

model = linear_model.LinearRegression()
model.fit(x, y)
# intercept_ と coef_ がそれぞれ w[0] と w[1:] になるように連結する
w = np.r_[model.intercept_, model.coef_]

print("w = ", w)
plot_reg((x,y), (x_val,y_val), w)

print("train: R^2 =", model.score(x, y), ", val: R^2 =", model.score(x_val, y_val))

In [ ]:
# 計画行列の作成と重回帰の Pipeline
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

degree = 3

polynomial_features = PolynomialFeatures(degree=degree, include_bias=False)
linear_regression = linear_model.LinearRegression()

# polynomial_features（命名：'pfeat'）に続けて linear_regression（命名：'reg'）を実行するモデル
model = Pipeline( [('pfeat', polynomial_features), ('reg', linear_regression)] )
model.fit(x, y)

# 'reg' の結果の intercept_ と coef_ がそれぞれ w[0] と w[1:] になるように連結する
w = np.r_[model.named_steps['reg'].intercept_, model.named_steps['reg'].coef_]

print("w = ", w)
plot_reg((x,y), (x_val,y_val), w, degree=degree)

print("train: MSE =", ((y - model.predict(x))**2).mean(), 
      ", val: MSE =", ((y_val - model.predict(x_val))**2).mean())

### 最小二乗解の様々な計算方法
まず，計画行列$\tilde{\boldsymbol X}$を作成します．

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
degree = 3
X = PolynomialFeatures(degree).fit_transform(x)

if degree < 4:
    with npopts():
        print("X =");    print(X)
        print("X.T.dot(X) =");    print(X.T.dot(X))

#### ★直接法
$\tilde{\boldsymbol w}^\star=\arg\min_{\tilde{\boldsymbol w}}\;\|{\boldsymbol y}-\tilde{\boldsymbol X}\tilde{\boldsymbol w}\|^2$

正規方程式：$\tilde{\boldsymbol X}^\top\!\!\tilde{\boldsymbol X}\,\;\tilde{\boldsymbol w}=\tilde{\boldsymbol X}^\top\!\!{\boldsymbol y}$

解の公式：$\tilde{\boldsymbol w}^\star=(\tilde{\boldsymbol X}^\top\!\!\tilde{\boldsymbol X})^{-1}\tilde{\boldsymbol X}^\top\!\!{\boldsymbol y}$

In [ ]:
from numpy import linalg
#from scipy import linalg

#### (a-1) 解の公式どおり逆行列を使う方法（https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html）
w = ''' linalg.inv を使いましょう '''

#### (a-2) 連立方程式の解法を使う方法（https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html）
w = ''' linalg.solve を使いましょう '''

#### (a-3) 一般逆行列を使う方法（https://numpy.org/doc/stable/reference/generated/numpy.linalg.pinv.html）
w = ''' linalg.pinv を使いましょう '''

#### (a-4) 最小二乗法を使う方法（https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html）
w = ''' linalg.lstsq を使いましょう '''

print("w =", w)
plot_reg((x,y), (x_val,y_val), w, degree=degree)

#### ☆反復法
最急降下法：$\tilde{\boldsymbol w}\gets \tilde{\boldsymbol w}-\alpha \frac{1}{n}\nabla_{\tilde{\boldsymbol w}}L(\tilde{\boldsymbol w})$

損失関数 $L(\tilde{\boldsymbol w})=\frac{1}{2n}\|{\boldsymbol y}-\tilde{\boldsymbol X}\tilde{\boldsymbol w}\|^2$

勾配 $\nabla_{\tilde{\boldsymbol w}}L(\tilde{\boldsymbol w})=-\frac{1}{n}\tilde{\boldsymbol X}^\top({\boldsymbol y}-\tilde{\boldsymbol X}\tilde{\boldsymbol w})$





In [ ]:
def lstsq_gd(X, y, lr = 1e-3, max_iter=1000):
    n = len(y)
    w = X.T.dot(y)
    loss_history = []
    iter = 0
    while iter < max_iter:
        delta = y - X.dot(w)
        w = ''' 最急降下法で w を更新してください '''
        loss_history.append(0.5*np.dot(delta, delta)/n)
        iter += 1
    return w, loss_history

w, lh = lstsq_gd(X, y, lr=1e-2, max_iter=10000)
print("w =", w)
plt.plot(np.arange(len(lh)),lh); plt.xscale('log'); plt.yscale('log')

plot_reg((x,y), (x_val,y_val), w, degree=degree)

--------
## 発展課題（任意）
画素を欠損させた顔画像から欠損前の画像を復元する回帰を試みましょう．モデルは3層ニューラルネットワークとします．

画像：[cropped Yale face database B](https://paperswithcode.com/dataset/extended-yale-b-1)，32x32画素，2414枚，38人．

    1. 欠損率を大きくすると，検証用画像の復元にどのような不具合が生じますか．

（ここに回答を書いてください）


    2. 活性化関数 'relu' とは何ですか．画像の復元において 'relu' や 'identity' を中間層に採用する利点はあるでしょうか．

（ここに回答を書いてください．

In [ ]:
#@title 画像をダウンロードし，訓練用画像 `X_train` と検証用 `X_val` に分けます．
import requests
import io

response = requests.get('https://github.com/tsakailab/prml/raw/master/datasets/YaleB_32x32.mat')
response.raise_for_status()
#data = np.load(io.BytesIO(response.content))
from scipy.io import loadmat
dataset = loadmat(io.BytesIO(response.content))

# split data into those for training (0.5), validation (0.5*0.5) and testing (0.5*0.5)
mybirthday = 19731115

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(dataset['fea'], dataset['gnd'].ravel(), train_size=0.8, random_state=mybirthday)
n_train, n_val = X_train.shape[0], X_val.shape[0]
n_pixels = X_train.shape[1]
print("#train =", n_train, ", #val =", n_val)

# zero-based indexing, shape (n,)
y_train, y_val = y_train-1, y_val-1

# scaling to [0,1]
X_train, X_val = X_train / 255., X_val / 255.

### ランダムに画素を欠損させます．
- 欠損率 `missing_pixel_rate` を設定してください．
- 欠損させた画素は赤で表示されます．

In [ ]:
# rate of missing pixels
missing_pixel_rate = 0.3    #### try 0.1, 0.3, 0.5 ####

In [ ]:
#@title 訓練画像を表示します．
import numpy as np

n_pixels_miss = np.ceil(n_pixels * missing_pixel_rate).astype(int)
p_miss = np.random.choice(n_pixels, n_pixels_miss, replace=False)

# put np.nan as missing pixels
X_in_train = X_train.copy()
X_in_train[:,p_miss] = np.nan
X_in_val = X_val.copy()
X_in_val[:,p_miss] = np.nan
p_obs = ~np.isnan(X_in_train[0,:])

import matplotlib.pyplot as plt
plt.cm.gray.set_bad(color='red')

print("Input images for training,", p_obs.sum(), "pixels are available for each")
fig, axes = plt.subplots(4, 6)
vmin, vmax = 0., 1.
for f, ax in zip(X_in_train, axes.ravel()):
    ax.matshow(f.reshape(32, 32).T, cmap=plt.cm.gray, vmin=vmin, vmax=vmax)
    ax.set_xticks(())
    ax.set_yticks(())
plt.show()

print("Output images each with", n_pixels, "pixels for training")
fig, axes = plt.subplots(4, 6)
vmin, vmax = 0., 1.
for f, ax in zip(X_train, axes.ravel()):
    ax.matshow(f.reshape(32, 32).T, cmap=plt.cm.gray, vmin=vmin, vmax=vmax)
    ax.set_xticks(())
    ax.set_yticks(())
plt.show()

### 3層ニューラルネットワークに学習させます．
- 入力層・中間層・出力層があります．
- 欠損画像の欠損していない画素値（画素数1024未満）を入力します．欠損していない正解画像（画素数32x32=1024）を予測したものを出力させたいです．
- `nnodes` は中間層のノード数です．
- [sklearn.neural_network.MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html)を使用します．

In [ ]:
# number of hidden layer nodes 
nnodes = 300        #### try 10 to 300 (e.g., 10, 30, 60, 100, 300) (caution: computationally expensive) ####

from sklearn.neural_network import MLPRegressor
model = MLPRegressor(hidden_layer_sizes=(nnodes,), activation='logistic',   # 'identity', 'relu', 'logistic'
                     verbose=True)

from time import time
t0 = time()
print("Training ..")
model.fit(X_in_train[:,p_obs], X_train)
print('done in %.2fs.' % (time() - t0))

print("train: R^2 =", model.score(X_in_train[:,p_obs], X_train), ", val: R^2 =", model.score(X_in_val[:,p_obs], X_val))

In [ ]:
#@title 検証画像の復元例を表示します．
plt.cm.gray.set_bad(color='red')
print("Input images for validation")
fig, axes = plt.subplots(4, 6)
vmin, vmax = 0., 1.
for f, ax in zip(X_in_val, axes.ravel()):
    ax.matshow(f.reshape(32, 32).T, cmap=plt.cm.gray, vmin=vmin, vmax=vmax)
    ax.set_xticks(())
    ax.set_yticks(())
plt.show()

print("Output images for validation")
X_val_pred = model.predict(X_in_val[:,p_obs])

fig, axes = plt.subplots(4, 6)
vmin, vmax = 0., 1.
for f, ax in zip(X_val_pred, axes.ravel()):
    ax.matshow(f.reshape(32, 32).T, cmap=plt.cm.gray, vmin=vmin, vmax=vmax)
    ax.set_xticks(())
    ax.set_yticks(())
plt.show()

print("Ground truth images for validation")
fig, axes = plt.subplots(4, 6)
vmin, vmax = 0., 1.
for f, ax in zip(X_val, axes.ravel()):
    ax.matshow(f.reshape(32, 32).T, cmap=plt.cm.gray, vmin=vmin, vmax=vmax)
    ax.set_xticks(())
    ax.set_yticks(())
plt.show()

In [ ]:
coefs = model.coefs_
print(coefs[0].shape, coefs[1].shape)

# Can you visualize coefs[1] with 300 images?



----
お疲れさまでした．